# <center>Aplicación de CNN para reconocimiento de Pokémon y generación de una interfaz explicativa</center>


## Datos Generales.

* Asignatura: Inteligencia Artificial en las Organizaciones
* Curso: 2023/2024
* Profesor: Agapito Ismael Ledezma Espino
* Grupo: 85

Alumnos: 
- Jonathan Jiménez Muñoz (100451132@alumnos.uc3m.es)
- Marta Palomo Velasco (100451041@alumnos.uc3m.es)
- Francisco Antonio Gallardo Fuentes (100451146@alumnos.uc3m.es)
- Yago Brotón Gutiérrez (100451322@alumnos.uc3m.es)

## Introducción

El objetivo de está práctica va a ser utilizar Redes Neuronales Convolucionales para clasificar imágenes de Pokémon y, una vez identificado dicho Pokémon, construir una pequeña interfaz donde se muestren estadísticas relevantes sobre el mismo. 

## Contexto

Cada año salen más y más Pokemon, habiendo actualmente más de 1000 especies diferentes, donde cada especie tiene n movimientos diferentes, 2/3 habilidades diferentes, naturalezas, tipos, etc. <br><br>
Es por ello que las personas junior que intentan competir profesionalmente en el juego de Pokemon, se ven abrumadas por la cantidad de información que tienen que procesar, y por ello cada vez hay más y más jugadores masters (+18 años) y menos senior y junior (menos de 18 años), dado que los jugadores que antes eran junior han cambiado de categoría y cada vez hay menos jugadores nuevos que se unan a la comunidad. <br><br>
Con esta práctica lo que queremos lograr es que, a partir de una simple imagen de un Pokemon, se muestre información relevante sobre el mismo, como por ejemplo, sus estadísticas base, sus habilidades, sus movimientos, etc, y con ello que los jugadores junior puedan aprender más rápido y tener algo más de motivación para entrar en la comunidad competitiva de Pokemon.

## Desarrollo de la práctica

Esta práctica va a estar dividida en varias fases:

1. Obtención de los datos: En esta fase vamos a obtener aquellas imágenes que vamos a utilizar para entrenar nuestro modelo, y vamos a preprocesarlas para que el modelo pueda entrenar con ellas. Además de obtener los datos usados para la interfaz. <br><br>
2. Preprocesado de los datos: Una vez obtengamos todos los datos/imágenes posibles, tendremos que hacer una limpieza (eliminar duplicados, imágenes que no correspondan al Pokemon que queremos, etc), con el objetivo de tener un dataset lo más limpio posible.<br><br>
3. Entrenamiento del modelo: Tras haber preprocesado los datos, el siguiente paso es entrenar el modelo. Para ello, como hemos dicho en la introducción, vamos a utilizar una Red Neuronal Convolucional (CNN), que es un tipo de red neuronal que se utiliza para clasificar imágenes.<br><br>
4. Selección del modelo: Una vez entrenado el modelo, tendremos que probar diferentes hiperparámetros y configuraciones de la red neuronal, con el objetivo de conseguir aquél modelo que mejor prediga al Pokemon que le pasemos.<br><br>
5. Desarrollo de la Interfaz: Finalmente, con el modelo ya entrenado y seleccionado, tendremos que desarrollar una interfaz que muestre aquella información (mejores movimientos, habilidades, estadísticas, etc) que consideremos relevantes para el jugador y que le ayuden a mejorar en el juego.

### 1. Obtención de los datos

Para obtener las imágenes y datos sobre los Pokémon, hemos utilizado diferentes fuentes de información, como por ejemplo:
Kaggle: 
- https://www.kaggle.com/datasets/abcsds/pokemon
- https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types
- https://www.kaggle.com/datasets/kvpratama/pokemon-images-dataset
- https://www.kaggle.com/datasets/arenagrenade/the-complete-pokemon-images-data-set

ImagenDex:
- https://pokemaster.es/imagendex/

Sin embargo, como vimos que la cantidad de imágenes era muy pequeña, decidimos crear un pequeño script que nos permitiera obtener imágenes a partir de Google Imágenes. <br>
Para ello, utilizamos la librería google_search para realizar búsquedas en Google Imágenes, la librería requests para realizar peticiones a las páginas web, y la librería BeautifulSoup para parsear el HTML de las páginas web y obtener las imágenes. <br>


In [ ]:
# Script para la búsqueda de imágenes en Google imágenes (descomentar si se quiere probar)

# Importamos las librerías necesarias
"""
import os
import requests
from bs4 import BeautifulSoup
from googlesearch import search

# Función para la descarga de imágenes
def download_all_images(query):
    image_links = set()

    # Searching for the query in Google Images
    search_query = query + " images"
    for j in search(search_query, num=10, stop=10, pause=2):
        if 'https://encrypted-tbn0.gstatic.com/images' in j:
            continue
        page = requests.get(j)
        soup = BeautifulSoup(page.content, "html.parser")

        # Extracting image links
        for raw_img in soup.find_all("img"):
            link = raw_img.get("src")
            if link and link.startswith("http"):
                image_links.add(link)

    # Create a directory for downloaded images
    if not os.path.exists(query):
        os.makedirs(query)

    # Download the images
    for i, link in enumerate(image_links):
        try:
            response = requests.get(link)
            file = open(os.path.join(query, f"{query}_{i+1}.jpg"), "wb")
            file.write(response.content)
            file.close()
        except Exception as e:
            print(f"Error: {e}")
            continue
    
    print(f"Downloaded {len(image_links)} images for {query}.")
    return


query = "squirtle in game"  # Introducir el término aquí.

download_all_images(query)
"""

Con todo esto, obtuvimos muchas imágenes para cada Pokémon. Sin embargo, decidimos quedarnos con los Pokémon que más imágenes tenían, debido a que consideramos que es mejor obtener un buen modelo que funcione para una cantidad reducida de especies, que crear un modelo que funcione para muchas especies pero no sea demasiado preciso. <br>

Por ello, los Pokémon que decidimos utilizar fueron los siguientes: Pikachu, Charmander, Charizard, Caterpie, Magikarp, Ratata, Geodude, Machop, Squirtle, Bulbasaur, Mew, Dragonite, Meowth, Lapras, Snorlax, Greninja, Rayquaza, Lucario, MrMime y Gengar (20 especies). <br>


### 2. Preprocesado de los datos
Debido a que utilizamos diferentes filtros en la búsqueda de imágenes para obtener las máximas posibles, nos salieron muchas imágenes duplicadas o que no se correspondían con el Pokémon que queríamos. <br>

Es por ello que debíamos hacer un preprocesado de los datos, para quedarnos con el conjunto de imágenes de cada Pokémon lo más limpio y homogéneo posible. <br>

Esto conyevó varias tareas: 
- Eliminación de imágenes no correspondientes al Pokémon que queríamos. Para ello simplemente una vez descargadas todas las imágenes, nos metíamos en la carpeta e íbamos eliminando una a una las imágenes que no pertenecían a dicho Pokémon.
- Eliminación de imágenes duplicadas. En este punto nos dimos cuenta de que hacerlo a mano iba a ser prácticamente imposible, por ello decidimos crear un pequeño script que comparase las imágenes entre sí y eliminase aquellas que fueran iguales, usando la libería Pilow.
- [Pendiente de preguntar] --> ¿Es necesario que tengan todas las imágenes las mismas dimensiones?

In [ ]:
# Script de eliminación de imágenes duplicadas (descomentar si se quiere probar)
"""
from PIL import Image
import os

# Ruta de la carpeta con las imágenes
carpeta = 'pokemon/Meowth'

# Diccionario para almacenar las imágenes idénticas
imagenes_ident = {}

# Recorriendo la carpeta y comparando cada imagen con las demás
for nombre_imagen1 in os.listdir(carpeta):
    ruta_imagen1 = os.path.join(carpeta, nombre_imagen1)
    if os.path.isfile(ruta_imagen1):
        imagen1 = Image.open(ruta_imagen1)
        for nombre_imagen2 in os.listdir(carpeta):
            ruta_imagen2 = os.path.join(carpeta, nombre_imagen2)
            if os.path.isfile(ruta_imagen2) and ruta_imagen1 != ruta_imagen2:
                imagen2 = Image.open(ruta_imagen2)
                if imagen1.size == imagen2.size and list(imagen1.getdata()) == list(imagen2.getdata()):
                    if nombre_imagen1 not in imagenes_ident:
                        imagenes_ident[nombre_imagen1] = [nombre_imagen2]
                    else:
                        imagenes_ident[nombre_imagen1].append(nombre_imagen2)

# Mostrar las imágenes idénticas encontradas
for imagen, imagenes_iguales in imagenes_ident.items():
    print(f"La imagen {imagen} es idéntica a: {', '.join(imagenes_iguales)}")
    
# Sacamos la imagen duplicada de la carpeta y la movemos a otra carpeta, dejando solamente una copia de la imagen
for imagen, imagenes_iguales in imagenes_ident.items():
    # si no existe la carpeta la creamos
    if not os.path.exists('pokemon/duplicadas'):
        os.makedirs('pokemon/duplicadas')
    # movemos la imagen a la carpeta duplicadas
    os.rename(carpeta + '/' + imagen, 'pokemon/duplicadas/' + imagen)
    # borramos las imagenes duplicadas
    for imagen_duplicada in imagenes_iguales:
        os.remove(carpeta + '/' + imagen_duplicada)
     
    
# Si no hay imágenes idénticas, mostrar un mensaje
if len(imagenes_ident) == 0:
    print("No hay imágenes idénticas.")

"""

Con esto, conseguimos tener un dataset lo más limpio posible, con el que poder entrenar nuestro modelo en el siguiente paso.